# Team Members 
<br>
Audrey Pohl | u0497618 | u0497618@utah.edu
<br>
Clarissa Seebhom | u0575630 | u0575630@utah.edu
<br>
Joseph Wirthlin | u0936690 | u0936690@utah.edu
<br>

# Background and Motivation 

Our group collectively enjoys outdoor activities in Salt Lake City. We ski, climb, hike, and camp in the wilderness surrounding Salt Lake City. For this reason, we value the Great Salt Lake for its contribution to the local ecosystem. With increased snowfall this winter, we are curious how much it will impact the level of the Salt Lake. Snowfall isn’t the only factor that can impact the water level. We also will analyze the factors of rainfall, total precipitation, and water usage. 

# Project Objectives

Do rainfall and snowfall have different effects on water levels of the GSL? Does precipitation or human factors have more of an impact on water levels of the GSL?


# Data

There are four sets of data that we are using for this project, precipitation, snow water equivalent, GDP, and population growth. The precipitation data for the Salt Lake Metropolitan Area comes from the National Weather Service. Snow water equivalent (SWE) is the amount of water in the Utah snowpack from the US Department of Agriculture. 

Precipitation - https://www.weather.gov/wrh/climate?wfo=slc <br>
SWE -  https://www.nrcs.usda.gov/Internet/WCIS/AWS_PLOTS/basinCharts/POR/WTEQ/assocHUCut3/state_of_utah.html <br>
GDP - https://fred.stlouisfed.org/series/UTNGSP <br>
Population data - https://www.macrotrends.net/states/utah/population#:~:text=The%20population%20of%20Utah%20in,a%201.53%25%20increase%20from%202018. <br>
